In [1]:
## Load Libs

In [2]:
# extra install
!conda install -y IProgress

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/computer_vision

  added / updated specs:
    - iprogress


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2021.10.8          |   py38h578d9bd_1         145 KB  conda-forge
    iprogress-0.4              |             py_0          12 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         157 KB

The following NEW packages will be INSTALLED:

  iprogress          conda-forge/noarch::iprogress-0.4-py_0

The following packages will be UPDATED:

  certifi                          2021.10.8-py38h578d9bd_0 --> 2021.10.8-py38h578d9bd_1



iprogress-0.4        | 12 KB     | ##################################### | 100% 
certifi-2021.10.8    | 145 KB    | ##################################### | 100% 
Preparing transacti

In [18]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from pytorch_lightning import LightningModule, seed_everything, Trainer
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim.swa_utils import AveragedModel, update_bn
from torchmetrics.functional import accuracy

In [4]:
seed_everything(7)

Global seed set to 7


7

In [6]:
PATH_DATASETS = os.environ.get('PATH_DATASETS', '.')
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64
NUM_WORKERS = int(os.cpu_count() / 2)

In [16]:
torch.cuda.device_count()

2

## Data Loaders

In [7]:
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    cifar10_normalization(),
])

test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    cifar10_normalization(),
])

cifar10_dm = CIFAR10DataModule(
    data_dir=PATH_DATASETS,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    #val_transforms=test_transforms,
)

In [8]:
def create_model():
    model = torchvision.models.resnet18(pretrained=False, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    model.maxpool = nn.Identity()
    return model

## Create Lightning Module

In [9]:
class LitResnet(LightningModule):

    def __init__(self, lr=0.05):
        super().__init__()

        self.save_hyperparameters()
        self.model = create_model()

    def forward(self, x):
        out = self.model(x)
        return F.log_softmax(out, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        self.log('train_loss', loss)
        return loss

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        if stage:
            self.log(f'{stage}_loss', loss, prog_bar=True)
            self.log(f'{stage}_acc', acc, prog_bar=True)

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, 'val')

    def test_step(self, batch, batch_idx):
        self.evaluate(batch, 'test')

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(
            self.parameters(),
            lr=self.hparams.lr,
            momentum=0.9,
            weight_decay=5e-4,
        )
        steps_per_epoch = 45000 // BATCH_SIZE
        scheduler_dict = {
            'scheduler': OneCycleLR(
                optimizer,
                0.1,
                epochs=self.trainer.max_epochs,
                steps_per_epoch=steps_per_epoch,
            ),
            'interval': 'step',
        }
        return {'optimizer': optimizer, 'lr_scheduler': scheduler_dict}

## Train Loop

In [15]:
AVAIL_GPUS

1

In [17]:
model = LitResnet(lr=0.05)
model.datamodule = cifar10_dm

trainer = Trainer(
    progress_bar_refresh_rate=10,
    max_epochs=30,
    #gpus=AVAIL_GPUS,
    gpus=[0,1],
    accelerator='dp',
    logger=TensorBoardLogger('lightning_logs/', name='resnet'),
    callbacks=[LearningRateMonitor(logging_interval='step')],
)

trainer.fit(model, cifar10_dm)
trainer.test(model, datamodule=cifar10_dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/opt/conda/envs/computer_vision/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11.2 M
---------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.696    Total estimated model params size (MB)


Global seed set to 7


Epoch 0:  81%|████████  | 160/197 [00:11<00:02, 14.22it/s, loss=1.54, v_num=7]
Validating: 0it [00:00, ?it/s]
Epoch 1:  81%|████████  | 160/197 [00:12<00:02, 13.21it/s, loss=1.33, v_num=7, val_loss=3.350, val_acc=0.169] 
Validating: 0it [00:00, ?it/s]
Epoch 2:  81%|████████  | 160/197 [00:12<00:02, 12.52it/s, loss=1.05, v_num=7, val_loss=4.540, val_acc=0.117] 
Validating: 0it [00:00, ?it/s]
Epoch 21:  41%|████      | 80/197 [01:39<02:23,  1.23s/it, loss=0.204, v_num=6, val_loss=1.980, val_acc=0.460]

Epoch 3:  81%|████████  | 160/197 [00:13<00:03, 12.26it/s, loss=0.901, v_num=7, val_loss=3.770, val_acc=0.187]
Validating: 0it [00:00, ?it/s]
Epoch 4:  81%|████████  | 160/197 [00:13<00:03, 11.51it/s, loss=0.718, v_num=7, val_loss=3.670, val_acc=0.154] 
Validating: 0it [00:00, ?it/s]
Epoch 5:  81%|████████  | 160/197 [00:14<00:03, 11.14it/s, loss=0.705, v_num=7, val_loss=3.450, val_acc=0.227]
Validating: 0it [00:00, ?it/s]
Epoch 6:  81%|████████  | 160/197 [00:15<00:03, 10.38it/s, loss=0.6

/opt/conda/envs/computer_vision/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(
/opt/conda/envs/computer_vision/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]



Testing: 100%|██████████| 40/40 [00:01<00:00, 33.57it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.9200999736785889, 'test_loss': 0.2666110694408417}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 40/40 [00:01<00:00, 25.60it/s]


/opt/conda/envs/computer_vision/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(


[{'test_loss': 0.2666110694408417, 'test_acc': 0.9200999736785889}]